In [2]:
!pip install torch-geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 9.0 MB/s eta 0:00:00


In [11]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.datasets import Planetoid

In [12]:
dataset = Planetoid(root=".", name="Cora")
data = dataset[0]

def accuracy(y_pred, y_true):
    """Calculate accuracy."""
    return torch.sum(y_pred == y_true) / len(y_true)

In [13]:
class GCN(torch.nn.Module):
    """Graph Convolutional Network"""
    def __init__(self, dim_in, dim_h, dim_out):
        super().__init__()
        self.gcn1 = GCNConv(dim_in, dim_h)
        self.gcn2 = GCNConv(dim_h, dim_out)

    def forward(self, x, edge_index):
        h = self.gcn1(x, edge_index)
        h = torch.relu(h)
        h = self.gcn2(h, edge_index)
        return F.log_softmax(h, dim=1)

    def fit(self, data, epochs):
        criterion = torch.nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(self.parameters(),
                                      lr=0.0001,
                                      weight_decay=5e-4)

        self.train()
        for epoch in range(epochs+1):
            optimizer.zero_grad()
            out = self(data.x, data.edge_index)
            loss = criterion(out[data.train_mask], data.y[data.train_mask])
            acc = accuracy(out[data.train_mask].argmax(dim=1),
                          data.y[data.train_mask])
            loss.backward()
            optimizer.step()

            if(epoch % 20 == 0):
                val_loss = criterion(out[data.val_mask], data.y[data.val_mask])
                val_acc = accuracy(out[data.val_mask].argmax(dim=1),
                                  data.y[data.val_mask])
                print(f'Epoch {epoch:>3} | Train Loss: {loss:.3f} | Train Acc:'
                      f' {acc*100:>5.2f}% | Val Loss: {val_loss:.2f} | '
                      f'Val Acc: {val_acc*100:.2f}%')

    @torch.no_grad()
    def test(self, data):
        self.eval()
        out = self(data.x, data.edge_index)
        acc = accuracy(out.argmax(dim=1)[data.test_mask], data.y[data.test_mask])
        return acc

In [14]:
# Create the Vanilla GNN model
gcn = GCN(dataset.num_features, 16, dataset.num_classes)
print(gcn)

# Train
gcn.fit(data, epochs=1000)

# Test
acc = gcn.test(data)
print(f'\nGCN test accuracy: {acc*100:.2f}%\n')

GCN(
  (gcn1): GCNConv(1433, 16)
  (gcn2): GCNConv(16, 7)
)
Epoch   0 | Train Loss: 1.933 | Train Acc: 20.71% | Val Loss: 1.93 | Val Acc: 19.20%
Epoch  20 | Train Loss: 1.902 | Train Acc: 30.71% | Val Loss: 1.92 | Val Acc: 24.00%
Epoch  40 | Train Loss: 1.867 | Train Acc: 41.43% | Val Loss: 1.90 | Val Acc: 33.00%
Epoch  60 | Train Loss: 1.827 | Train Acc: 51.43% | Val Loss: 1.88 | Val Acc: 41.60%
Epoch  80 | Train Loss: 1.782 | Train Acc: 58.57% | Val Loss: 1.85 | Val Acc: 46.40%
Epoch 100 | Train Loss: 1.732 | Train Acc: 67.14% | Val Loss: 1.82 | Val Acc: 50.60%
Epoch 120 | Train Loss: 1.679 | Train Acc: 75.00% | Val Loss: 1.78 | Val Acc: 54.40%
Epoch 140 | Train Loss: 1.626 | Train Acc: 81.43% | Val Loss: 1.75 | Val Acc: 58.60%
Epoch 160 | Train Loss: 1.573 | Train Acc: 85.71% | Val Loss: 1.72 | Val Acc: 61.60%
Epoch 180 | Train Loss: 1.522 | Train Acc: 88.57% | Val Loss: 1.69 | Val Acc: 65.20%
Epoch 200 | Train Loss: 1.472 | Train Acc: 90.71% | Val Loss: 1.66 | Val Acc: 67.60%
Epoch